# Manage scRNA-seq datasets

This illustrates how to manage scRNA-seq datasets in absence of a custom schema!

In [ ]:
!lamin init --storage ./test-scrna --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

ln.settings.verbosity = 3  # show hints

In [ ]:
ln.track()

## Preparation: registries

Let's assume that this is not the first time we work with experimental entities, and hence, our registries are already pre-populated:

In [ ]:
# assume prepared registries

# strain
lb.ExperimentalFactor.from_bionty(ontology_id="EFO:0004472").save()

# developmental stage
lb.ExperimentalFactor.from_bionty(ontology_id="EFO:0001272").save()

# tissue
lb.Tissue.from_bionty(ontology_id="UBERON:0001542").save()

# cell types
ln.save(lb.CellType.from_values(["CL:0000115", "CL:0000738"], "ontology_id"))

In [ ]:
ln.view(schema="bionty")

## Detmar22: Mouse Lymph Node

We're working with mouse data:

In [ ]:
lb.settings.species = "mouse"

Let's look at a scRNA-seq count matrix in form of an `AnnData` object:

In [ ]:
adata = ln.dev.datasets.anndata_mouse_sc_lymph_node()

In [ ]:
adata

Let's have a look in the annotations:

In [ ]:
adata.obs.columns

The column names are a bit lengthy, let's abbreviate them:

In [ ]:
adata.obs.columns = adata.obs.columns.str.replace(
    "Sample Characteristic", ""
).str.replace(" Ontology Term", "ontology_id")

When we create a file object from an AnnData, we'll automatically link its feature sets and get information about unmapped categories:

In [ ]:
file = ln.File.from_anndata(
    adata, description="Detmar22", var_ref=lb.Gene.ensembl_gene_id
)

In [ ]:
file.save()

The file now has two linked feature sets:

In [ ]:
file.feature_sets.df()

Let's also link observational metadata:

In [ ]:
strain = lb.ExperimentalFactor.select(ontology_id="EFO:0004472").one()
file.experimental_factors.add(strain)
dev_stage = lb.ExperimentalFactor.select(ontology_id="EFO:0001272").one()
file.experimental_factors.add(dev_stage)
cell_types = lb.CellType.from_values(["CL:0000115", "CL:0000738"], "ontology_id")
file.cell_types.set(cell_types)

The file is now queryable by everything we linked:

In [ ]:
file.describe()

## Human immune cells: Conde22

In [ ]:
lb.settings.species = "human"

In [ ]:
conde22 = ln.dev.datasets.anndata_human_immune_cells()

In [ ]:
file = ln.File.from_anndata(
    conde22, description="Conde22", var_ref=lb.Gene.ensembl_gene_id
)
file.save()

The file has the following linked features:

In [ ]:
file.feature_sets.df()

Let's now link observational metadata.

In [ ]:
cell_types = lb.CellType.from_values(conde22.obs.cell_type, field="name")
ln.save(cell_types)
file.cell_types.set(cell_types)

In [ ]:
efs = lb.ExperimentalFactor.from_values(conde22.obs.assay, field="name")
ln.save(efs)
file.experimental_factors.set(efs)

In [ ]:
tissues = lb.Tissue.from_values(conde22.obs.tissue, field="name")
ln.save(tissues)
file.tissues.set(tissues)

As neither the core schema nor `lnschema_bionty` have a `Donor` table, we're using `Label` to track donor ids:

In [ ]:
donor = ln.Label(name="donor", description="Parent label for all donor labels")
donor.save()
donors = ln.Label.from_values(conde22.obs["donor_id"])
ln.save(donors)
[d.parents.add(donor) for d in donors]
file.labels.set(donors)
donor.children.df()

In [ ]:
file.describe()

## A less well curated dataset

Let's now consider a dataset with less-well curated features:

In [ ]:
pbcm68k = ln.dev.datasets.anndata_pbmc68k_reduced()

We see that this dataset is indexed by gene symbols: 

In [ ]:
pbcm68k.var.index

Because gene symbols don't uniquely characterize an Ensembl ID, we're linking more feature records to this file than columns in the `AnnData`.

```{tip}

Use Ensembl Gene IDs rather than gene Symbols to index genes.

```

In [ ]:
ln.File.from_anndata(
    pbcm68k, description="10x reference pbmc68k", var_ref=lb.Gene.symbol
).save()

## View the database content

In [ ]:
ln.view()

In [ ]:
!lamin delete test-scrna
!rm -r ./test-scrna